In [1]:
import pandas as pd
import numpy as np

In [14]:
data = pd.read_csv(r"C:\Users\luyil\ihealth\datasets\preproc\services\payers.csv")
data.shape

(6872085, 26)

In [4]:
data.head()

,Unnamed: 0.1,Unnamed: 0,payer,insurer_paid_amt_mean,insurer_paid_amt_sum,member_paid_amt_mean,member_paid_amt_sum,total_paid_amt_mean,total_paid_amt_sum,row_id,...,unique_member_cnt,svc_code,svc_code_type,svc_code_ctg,svc_code_ctg_desc,clm_cnt,member_months,total_paid_amt_median,insurer_paid_amt_median,member_paid_amt_median
0,15330,0,MEDICARE,0.00,0.00,0.00,0.00,0.00,0.00,3127575.0,...,15.0,G8482,CPT/HCPCS,999.0,HCPCS Level II codes not classified,21.0,180.0,0.00,0.00,0.00
1,15331,1,COMMERCIAL,258.94,3366.29,71.67,931.73,330.61,4298.02,2888025.0,...,13.0,76705,CPT/HCPCS,196.0,Ultrasound - Abdomen/retroperitoneum - Diagnostic,24.0,137.0,328.00,284.33,43.26
2,15332,2,MINNESOTA HEALTH CARE PROGRAMS,225.33,28392.81,0.00,0.00,225.33,28392.81,2878775.0,...,126.0,A0425,CPT/HCPCS,239.0,"Transportation - Patient, provider, equipment",205.0,1321.0,81.83,81.83,0.00
3,15333,3,COMMERCIAL,1311.12,14422.37,32.76,360.45,1343.89,14782.82,3142946.0,...,11.0,C8929,CPT/HCPCS,193.0,Ultrasound - Heart (echocardiogram) - Diagnostic,11.0,132.0,1681.67,1674.97,0.00
4,15334,4,MEDICARE,2922.00,46752.03,1102.31,17637.01,4024.31,64389.04,3127585.0,...,16.0,J0690,CPT/HCPCS,240.0,"Medications (injections, infusions, other forms)",22.0,192.0,0.00,0.00,0.00


In [15]:
data = data.drop(["Unnamed: 0.1","Unnamed: 0"],axis=1)
data.shape

(6872085, 24)

In [ ]:
# feature decoding
def decoding(data):
    for idx, record in data.iterrows():
        record["service"] = ""
        if record["svc_code_type"] == "REV":
            record["service"] = "Non-medical"
        else:
            if record["svc_code"][0].isdigit():
                # CPT code
                i_code = int(record["svc_code"])
                if i_code>=99201 and i_code<=99499:
                    record["service"] = "Evaluation and Management"
                elif (i_code>=100 and i_code<=1999) or (i_code>=99100 and i_code<=99140):
                    record["service"] = "Anesthesia"
                elif i_code>=10021 and i_code<=69990:
                    record["service"] = "Surgery"
                elif i_code>=70010 and i_code<=79999:
                    record["service"] = "Radiology"
                elif i_code>=80047 and i_code<=89398:
                    record["service"] = "Patheology and Laboratory"
                else:
                    record["service"] = "Medicine"
            else:
                # HPCPC code
                c_code = record["svc_code"][:1]
                n_code = int(record["svc_code"][1:])
                # A, B, C, E, G, H, J, K, L, M, P, Q, R, S, T, U, V
                # 请在此处编写对HCPCS code的识别实现逻辑
                if c_code == "A":
                    if n_code <= 9999:
                        record["service"] = "Ambulance and Other Transport Services and Supplies"
                    elif 2001 <= n_code <= 2013:
                        record["service"] = "Matrix for Wound Management"
                    elif n_code == 4100:
                        record["service"] = "Skin Substitute Device"
                    elif 4206 <= n_code <= 8004:
                        record["service"] = "Medical And Surgical Supplies"
                    elif 9150 <= n_code <= 9999:
                        record["service"] = "Administrative, Miscellaneous and Investigational"
                elif c_code == "B":
                    if 4034 <= n_code <= 9999:
                        record["service"] = "Enteral and Parenteral Therapy"
                elif c_code == "C":
                    if 1052 <= n_code <= 1062:
                        record["service"] = "Other Therapeutic Procedures"
                    elif 1713 <= n_code <= 9899:
                        record["service"] = "Outpatient PPS"
                elif c_code == "E":
                    if 100 <= n_code <= 8002:
                        record["service"] = "Durable Medical Equipment"
                elif c_code == "G":
                    if 8 <= n_code <= 9987:
                        record["service"] = "Procedures / Professional Services"
                elif c_code == "H":
                    if 1 <= n_code <= 2037:
                        record["service"] = "Alcohol and Drug Abuse Treatment"
                elif c_code == "J":
                    if 120 <= n_code <= 8999:
                        record["service"] = "Drugs Administered Other than Oral Method"
                    elif 9000 <= n_code <= 9999:
                        record["service"] = "Chemotherapy Drugs"
                elif c_code == "K":
                    if 1 <= n_code <= 900:
                        record["service"] = "Durable medical equipment (DME) Medicare administrative contractors (MACs)"
                    elif 1001 <= n_code <= 1034:
                        record["service"] = "Components, Accessories and Supplies"
                elif c_code == "L":
                    if 112 <= n_code <= 4631:
                        record["service"] = "Orthotic Procedures and services"
                    elif 5000 <= n_code <= 9900:
                        record["service"] = "Prosthetic Procedures"
                elif c_code == "M":
                    if 75 <= n_code <= 301:
                        record["service"] = "Miscellaneous Medical Services"
                    elif 1000 <= n_code <= 1071:
                        record["service"] = "Screening Procedures"
                    elif 1106 <= n_code <= 1143:
                        record["service"] = "Episode of Care"
                    elif 1146 <= n_code <= 1149:
                        record["service"] = "Other Services"
                elif c_code == "P":
                    if 2028 <= n_code <= 9615:
                        record["service"] = "Pathology and Laboratory Services"
                elif c_code == "Q":
                    if 35 <= n_code <= 9992:
                        record["service"] = "Temporary Codes"
                elif c_code == "R":
                    if 70 <= n_code <= 76:
                        record["service"] = "Diagnostic Radiology Services"
                elif c_code == "S":
                    if 12 <= n_code <= 9999:
                        record["service"] = "Temporary National Codes (Non-Medicare)"
                elif c_code == "T":
                    if 1000 <= n_code <= 5999:
                        record["service"] = "National Codes Established for State Medicaid Agencies"
                elif c_code == "U":
                    if 1 <= n_code <= 5:
                        record["service"] = "Coronavirus Diagnostic Panel"
                elif c_code == "V":
                    if 2020 <= n_code <= 2799:
                        record["service"] = "Vision Services"
                    elif 5008 <= n_code <= 5364:
                        record["service"] = "Hearing Services"
                else:
                    record["service"] = "Unknown"

In [18]:
def decode_svc_code(svc_code):
    # 判断是否为数字的函数
    def is_digit(n):
        try:
            int(n)
            return True
        except ValueError:
            return False

    # CPT代码的处理逻辑
    def decode_cpt_code(i_code):
        if 99201 <= i_code <= 99499:
            return "Evaluation and Management"
        elif (100 <= i_code <= 1999) or (99100 <= i_code <= 99140):
            return "Anesthesia"
        elif 10021 <= i_code <= 69990:
            return "Surgery"
        elif 70010 <= i_code <= 79999:
            return "Radiology"
        elif 80047 <= i_code <= 89398:
            return "Pathology and Laboratory"
        else:
            return "Medicine"

    # HCPCS代码的服务类型映射
    hcpcs_service_mapping = {
        'A': [("0021-0999", "Ambulance and Other Transport Services and Supplies"),
            ("2001-2013", "Matrix for Wound Management (Placental, Equine, Synthetic)"),
            ("4100", "Skin Substitute Device"),
            ("4206-8004", "Medical And Surgical Supplies"),
            ("9150-9999", "Administrative, Miscellaneous and Investigational")],
        'B': [("4034-9999", "Enteral and Parenteral Therapy")],
        'C': [("1052-1062", "Other Therapeutic Procedures"),
            ("1713-9899", "Outpatient PPS")],
        'E': [("0100-8002", "Durable Medical Equipment")],
        'G': [("0008-9987", "Procedures / Professional Services")],
        'H': [("0001-2037", "Alcohol and Drug Abuse Treatment")],
        'J': [("0120-8999", "Drugs Administered Other than Oral Method"),
            ("9000-9999", "Chemotherapy Drugs")],
        'K': [("0001-0900", "Durable medical equipment (DME) Medicare administrative contractors (MACs)"),
            ("1001-1034", "Components, Accessories and Supplies")],
        'L': [("0112-4631", "Orthotic Procedures and services"),
            ("5000-9900", "Prosthetic Procedures")],
        'M': [("0075-0301", "Miscellaneous Medical Services"),
            ("1000-1071", "Screening Procedures"),
            ("1106-1143", "Episode of Care"),
            ("1146-1149", "Other Services")],
        'P': [("2028-9615", "Pathology and Laboratory Services")],
        'Q': [("0035-9992", "Temporary Codes")],
        'R': [("0070-0076", "Diagnostic Radiology Services")],
        'S': [("0012-9999", "Temporary National Codes (Non-Medicare)")],
        'T': [("1000-5999", "National Codes Established for State Medicaid Agencies")],
        'U': [("0001-0005", "Coronavirus Diagnostic Panel")],
        'V': [("2020-2799", "Vision Services"),
            ("5008-5364", "Hearing Services")]
    }


    if svc_code[0].isdigit():  # CPT code logic
        if is_digit(svc_code):
            i_code = int(svc_code)
            return decode_cpt_code(i_code)
        else:
            return "Unknown"
    else:  # HCPCS code logic
        c_code = svc_code[0]
        n_code_str = svc_code[1:]
        if is_digit(n_code_str):
            n_code = int(n_code_str)
            for start_end, service in hcpcs_service_mapping.get(c_code, []):
                if "-" in start_end:
                    start, end = map(int, start_end.split("-"))
                    if start <= n_code <= end:
                        return service
                elif int(start_end) == n_code:
                    return service
        return "Unknown"

def decoding(data):
    data['service'] = data['svc_code'].apply(decode_svc_code)
    return data



In [19]:
data = decoding(data)
data["service"].head()

0                   Procedures / Professional Services
1                                            Radiology
2    Ambulance and Other Transport Services and Sup...
3                                       Outpatient PPS
4            Drugs Administered Other than Oral Method
Name: service, dtype: object

In [20]:
data["service"].value_counts()

service
Pathology and Laboratory                                                      1681219
Medicine                                                                      1235350
Anesthesia                                                                     650119
Radiology                                                                      625426
Evaluation and Management                                                      602648
Surgery                                                                        533000
Drugs Administered Other than Oral Method                                      421841
Procedures / Professional Services                                             192014
Medical And Surgical Supplies                                                  188328
Temporary Codes                                                                 77889
Durable Medical Equipment                                                       77858
Ambulance and Other Transport Services and Sup

In [22]:
data.to_csv(r"C:\Users\luyil\ihealth\datasets\preproc\decoded_data.csv")
print("successed!")

successed!
